In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_multi_class_classifier import MultiClassDNNClassifer
from sklearn.preprocessing import LabelEncoder
from random import randint
from keras import callbacks
from keras import losses
from visualizer import *
from keras.utils import np_utils
from sklearn.metrics import classification_report

2023-01-16 17:35:56.717424: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-16 17:35:56.717541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-16 17:35:56.717553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
timeframe_in_ms = {"15m":15*60*1000,"1h":60*60*1000,"1d":24*60*60*1000}

symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

tp = 0.05
sl = 0.025
md = 14


data = pd.read_csv("../data/{}-{}.csv".format(
    symbol,trade_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

granular_data = pd.read_csv("../nocommit/{}-{}.csv".format(
    symbol,granular_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

fm: FeatureManager = FeatureManager()
fm.build_feature(data=data,lags=5)

fm.prepare_trade_forward_data(
    data=fm.df,
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
    granular_data=granular_data,
    timeframe_in_ms=timeframe_in_ms[trade_tf]
)

Processing 1/14
Processing 2/14
Processing 3/14
Processing 4/14
Processing 5/14
Processing 6/14
Processing 7/14
Processing 8/14
Processing 9/14
Processing 10/14
Processing 11/14
Processing 12/14
Processing 13/14
Processing 14/14


In [3]:
classifier = MultiClassDNNClassifer()
classifier.prepare_data(data = fm.df,cols = fm.cols,random_state=randint(0,100),target_col="trade_signal")


Train = 80963, Val = 17349, Test = 17350, All = 115662


In [4]:
classifier.configure(hu = 100, dropout=True, class_num = 3, input_dim=len(fm.cols))
classifier.run(gpu = True,epochs = 200, patience = 5)
classifier.model.evaluate(classifier.x_test,classifier.y_test)

2023-01-16 17:38:36.972513: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-01-16 17:38:36.972592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: andy-GA-970A-D3
2023-01-16 17:38:36.972617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: andy-GA-970A-D3
2023-01-16 17:38:36.972875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 520.61.5
2023-01-16 17:38:36.972950: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2023-01-16 17:38:36.972975: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 520.61.5


Epoch 1/200

Epoch 1: val_loss improved from inf to 0.99804, saving model to ../data/model_dnn_checkpoint.h5
2531/2531 - 9s - loss: 1.6276 - accuracy: 0.4354 - val_loss: 0.9980 - val_accuracy: 0.4921 - 9s/epoch - 4ms/step
Epoch 2/200

Epoch 2: val_loss improved from 0.99804 to 0.98482, saving model to ../data/model_dnn_checkpoint.h5
2531/2531 - 8s - loss: 1.5262 - accuracy: 0.4763 - val_loss: 0.9848 - val_accuracy: 0.5033 - 8s/epoch - 3ms/step
Epoch 3/200

Epoch 3: val_loss improved from 0.98482 to 0.97239, saving model to ../data/model_dnn_checkpoint.h5
2531/2531 - 9s - loss: 1.5042 - accuracy: 0.4892 - val_loss: 0.9724 - val_accuracy: 0.5139 - 9s/epoch - 3ms/step
Epoch 4/200

Epoch 4: val_loss improved from 0.97239 to 0.96266, saving model to ../data/model_dnn_checkpoint.h5
2531/2531 - 9s - loss: 1.4874 - accuracy: 0.4970 - val_loss: 0.9627 - val_accuracy: 0.5200 - 9s/epoch - 3ms/step
Epoch 5/200

Epoch 5: val_loss improved from 0.96266 to 0.95410, saving model to ../data/model_dnn_c

[0.6391882300376892, 0.7423054575920105]

In [5]:
classifier.print_classification_report()

              precision    recall  f1-score   support

           0       0.73      0.60      0.66      5742
           1       0.81      0.74      0.77      5826
           2       0.70      0.89      0.78      5782

    accuracy                           0.74     17350
   macro avg       0.75      0.74      0.74     17350
weighted avg       0.75      0.74      0.74     17350



In [ ]:
pd.DataFrame(classifier.y_test).value_counts()

In [ ]:
pd.DataFrame(classifier.pred_class).value_counts()

In [ ]:
acc_list = []
loss_list = []
laps = 5

for i in range (0,laps):
    print("\n======= Lap {} =======".format(i+1))
    callbacks.backend.clear_session()
    
    classifier = MultiClassDNNClassifer()
    
    classifier.configure(
        hu = 100, 
        dropout=True, 
        input_dim=len(fm.cols),
        class_num=3
    )
    
    classifier.prepare_data(
        data = fm.df,
        cols = fm.cols,
        shuffle = True, 
        random_state=i+1,
        target_col="trade_signal"
    )
    
    classifier.run(gpu = True,epochs=100, patience=7)
    classifier.visualize_loss()
    classifier.visualize_accuracy()
    
    test_results = classifier.model.evaluate(
        classifier.x_test,
        classifier.y_test
    )

    acc_list.append(test_results[1])
    loss_list.append(test_results[0])

acc_arr = np.array(acc_list)
loss_arr = np.array(loss_list)

print("\n======")
print("Accuracy mean: {}, std: {}".format(acc_arr.mean(),acc_arr.std()))
print("Loss mean: {}, std: {}".format(loss_arr.mean(),loss_arr.std()))
print("======")

In [ ]:
pred = pd.DataFrame(data=classifier.pred_prob,columns=["No trade","Long","Short"])
threshold = 0.5
filtered= pred.loc[(pred["Long"]>threshold)|(pred["Short"]>threshold)]
len(filtered)/len(pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
val_counts = data["trade_signal"].value_counts().sort_index()
weights = val_counts.max()/val_counts
data_lst = []
for i in val_counts.index:
    data_by_val = data[data["trade_signal"]==i].sample(replace=True,frac=weights[i],random_state=randint(0,100))
    data_lst.append(data_by_val)
data = pd.concat(data_lst)

In [ ]:
classifier.pred_class

In [ ]:
classifier.pred_prob

In [ ]:
pd.DataFrame(classifier.pred_class).value_counts()

In [ ]:
y_pred_int = np.argmax(classifier.pred_prob, axis=1)
y_test_int = np.argmax(classifier.y_test,axis = 1)
print(classification_report(y_test_int, y_pred_int))

In [ ]:
pd.DataFrame(classifier.y_test).value_counts()